In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation

In [39]:

# doc = nlp("Bake for 55 to 60 minutes, or until filling is set")
doc = nlp('In a medium bowl, mix together cookie crumbs, 3 tablespoons sugar, and melted butter')
for token in doc:
    print(token, token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
#     print(token.head, token.text)
displacy.render(doc, style="dep")


In In ADP IN prep Xx True True <generator object at 0x000001F948E6A040> mix
a a DET DT det x True True <generator object at 0x000001F948E6A040> bowl
medium medium ADJ JJ amod xxxx True False <generator object at 0x000001F948E6A040> bowl
bowl bowl NOUN NN pobj xxxx True False <generator object at 0x000001F948E6A040> In
, , PUNCT , punct , False False <generator object at 0x000001F948E6A040> mix
mix mix VERB VB ROOT xxx True False <generator object at 0x000001F948E6A040> mix
together together ADV RB advmod xxxx True True <generator object at 0x000001F948E6A040> mix
cookie cookie NOUN NN compound xxxx True False <generator object at 0x000001F948E6A040> crumbs
crumbs crumbs NOUN NNS dobj xxxx True False <generator object at 0x000001F948E6A040> mix
, , PUNCT , punct , False False <generator object at 0x000001F948E6A040> crumbs
3 3 NUM CD nummod d False False <generator object at 0x000001F948E6A040> tablespoons
tablespoons tablespoons NOUN NNS compound xxxx True False <generator object at 0x

In [3]:
def find_head(key, d):
    if key not in d:
        return key
    else:
        find_head(d[key], d)

In [41]:
# Methods Function

def find_methods(step):
    newstep = step

    doc = nlp(newstep)
    methods = {}
    timescale = {}
    times = {}
    connector = {}
    for i, entity in enumerate(doc):
        if entity.pos_ == "VERB":  #and str(entity.head) == str(entity.text)
            methods[entity] = {'times':[], 'scale':None}
        if entity.dep_ == "pobj":
            timescale[entity] = entity.head
        if entity.pos_ == "NUM" and (entity.dep_ == 'quantmod' or entity.dep_ == 'nummod'):
            times[entity] = entity.head
#         for m in methods:
#             if entity.head == m and str(entity.text) != str(entity.head):
#                 connector[entity] = m
        if str(entity.text) != str(entity.head):
            connector[entity] = entity.head

    for m in methods:
        if m in connector:
            n = connector[m]
            if n not in methods:
                connector[n] = m
                connector.pop(m)
    
    for scale in timescale:
        while timescale[scale] not in methods:
            if timescale[scale] not in connector:
                print("Error in dependency parsing")
                break
            timescale[scale] = connector[timescale[scale]]
            
    for time in times:
        if times[time] in times:
            times[time] = times[times[time]]
    
    for s in timescale:
        for t in times:
            if times[t] == s:
                methods[timescale[s]]['times'].append(float(str(t)))
        methods[timescale[s]]['scale'] = str(s)
        
    M = {}
    for m in methods:
        if methods[m]['scale'] == 'seconds':
            methods[m]['times'] = [t/60 for t in methods[m]['times']]
            methods[m]['scale'] = 'minutes'
        elif methods[m]['scale'] == 'hours':
            methods[m]['times'] = [t*60 for t in methods[m]['times']]
            methods[m]['scale'] = 'minutes'
            
        val = methods[m]
        M[str(m).lower()] = val
    
    ## get indices of times, pobj (minutes, hours, days, etc), and method, and use distance to see which ones belong to which one.

    return M

In [45]:
m = find_methods('In a medium bowl, Mix together cookie crumbs, 3 tablespoons sugar, and melted butter for 35 to 45 minutes')
print(m)

{'mix': {'times': [35.0, 45.0], 'scale': 'minutes'}}


SyntaxError: invalid syntax (<ipython-input-30-d94445d8f978>, line 1)

In [7]:
print(m)
print(t)

{'filling': 2, 'minutes': 5}
{'55': 8, 'Bake': 10}
